In [ ]:
%%capture

import calitp_data_analysis.magics

import warnings
warnings.filterwarnings('ignore')

import _report_utils as utils
import great_tables as gt
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html



In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
name = "Bay Area 511 AC Transit Schedule"

In [ ]:
# %%capture_parameters
# name

In [ ]:
sched_vp_df = utils.load_schedule_vp_metrics(name)

In [ ]:
gtfs_schedule_keys = list(sched_vp_df.schedule_gtfs_dataset_key.unique())

In [ ]:
monthly_df = utils.summarize_monthly("2023", name)

# Overview for {name}

## Operator Overview - To Come

## Route-Direction Monthly Overview
Statistics presented on the <b>route</b> level. Scroll down to filter for a specific route. 

In [ ]:
utils.single_bar_chart_dropdown(
    monthly_df,
    "day_type",
    "ttl_service_hours",
    "time_of_day",
    "Total Service Hours across all Routes per Month in 2023",
    "full_date",
)

In [ ]:
utils.trips_by_gtfs(sched_vp_df)

In [ ]:
utils.filtered_route(sched_vp_df)